# Import & Config

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

### Load Data

In [38]:
# Read the CSV file
df_raw = pd.read_csv('../data/raw/creditscore_data_raw.csv')
print("Dataset shape:", df_raw.shape)
print("\nDataset info:")
df_raw.info()

Dataset shape: (100000, 28)

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Ty

/var/folders/th/r1yk0vpn0rz7y_4vcvcbqy8h0000gn/T/ipykernel_37039/24460700.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('../data/raw/creditscore_data_raw.csv')


### Columns Validation

In [39]:
# Get basic statistics of the dataset
print("Dataset description:")
print(df_raw.describe())

print("\nNull values in each column:")
print(df_raw.isnull().sum())

Dataset description:
       Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  \
count           84998.000000      100000.000000     100000.00000   
mean             4194.170850          17.091280         22.47443   
std              3183.686167         117.404834        129.05741   
min               303.645417          -1.000000          0.00000   
25%              1625.568229           3.000000          4.00000   
50%              3093.745000           6.000000          5.00000   
75%              5957.448333           7.000000          7.00000   
max             15204.633333        1798.000000       1499.00000   

       Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
count  100000.000000        100000.000000          98035.000000   
mean       72.466040            21.068780             27.754251   
std       466.422621            14.860104            193.177339   
min         1.000000            -5.000000              0.000000   
25%         8.000000           

Quando olhamos para o resultado do .describe.show vemos aqui algumas oportunidades de tratamento.
Colunas como "Age" merecem atenção, variando de -500 a 995 com uma média e desvio padrão fora da realidade.
Não só ela, temos também colunas com "_" após ou antes o valor, desvios padrões e médias que indicam presença de outliers.

### Data Transform

Decidimos por eliminar colunas que não vão influenciar no modelo e estão apenas aumentando o número de dados, como "Name" e "SSN".
Aplicamos também substituições de valores que achamos válidas, retiramos o "_", limitando valores que estão de fato fora da realidade.

In [40]:
def data_transform(df):
    # Create a copy of the dataframe
    df = df.copy()
    
    # Drop unnecessary columns
    df = df.drop(['Name', 'SSN'], axis=1)
    
    # Replace placeholder values
    df['Occupation'] = df['Occupation'].replace('_______', 'N/A')
    
    # Handle Monthly_Inhand_Salary
    df['Monthly_Inhand_Salary'] = pd.to_numeric(df['Monthly_Inhand_Salary'], errors='coerce')
    df['Annual_Income'] = pd.to_numeric(df['Annual_Income'].astype(str).str.replace('_', ''), errors='coerce')
    df.loc[df['Monthly_Inhand_Salary'].isna(), 'Monthly_Inhand_Salary'] = df['Annual_Income'] / 12
    
    # Clean and convert numeric columns - handle string values first
    numeric_cols = ['Age', 'Num_of_Loan', 'Monthly_Balance', 'Num_of_Delayed_Payment', 
                   'Changed_Credit_Limit', 'Amount_invested_monthly']
    
    for col in numeric_cols:
        # Convert to string first to handle any non-string values
        df[col] = df[col].astype(str)
        # Replace underscores and convert to numeric
        df[col] = pd.to_numeric(df[col].str.replace('_', ''), errors='coerce')
    
    # Replace special values
    df['Credit_Mix'] = df['Credit_Mix'].replace('_', 'N/A')
    df['Payment_Behaviour'] = df['Payment_Behaviour'].replace('!@9#%8', 'N/A')
    
    # Handle age outliers
    df.loc[(df['Age'] > 100) | (df['Age'] <= 0), 'Age'] = np.nan
    
    return df

# Apply transformations
df_raw = data_transform(df_raw)

# Check the results
print("\nTransformed dataset info:")
df_raw.info()

print("\nCheck for any remaining underscores in data:")
for column in df_raw.columns:
    if df_raw[column].dtype == 'object':  # Only check string columns
        has_underscore = df_raw[column].astype(str).str.contains('_').any()
        if has_underscore:
            print(f"Column {column} still contains underscores")
            print(df_raw[df_raw[column].astype(str).str.contains('_')][column].unique())

print("\nSample of transformed data:")
print(df_raw.head())


Transformed dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Age                       97224 non-null   float64
 4   Occupation                100000 non-null  object 
 5   Annual_Income             100000 non-null  float64
 6   Monthly_Inhand_Salary     100000 non-null  float64
 7   Num_Bank_Accounts         100000 non-null  int64  
 8   Num_Credit_Card           100000 non-null  int64  
 9   Interest_Rate             100000 non-null  int64  
 10  Num_of_Loan               100000 non-null  int64  
 11  Type_of_Loan              88592 non-null   object 
 12  Delay_from_due_date       100000 non-null  int64  
 13  Num_of_Delayed_Pay

# Tratamento de coluna Credit_History_Age

Para podermos trabalhar com a coluna Credit_History_Age é necessário transformá-la em valores: usando uma fórmula simples de years * 12 + months

In [41]:
def convert_credit_history_age_to_months(df, source_col='Credit_History_Age', target_col='Credit_History_Age_Months'):
    """
    Convert credit history from 'XX Years and YY Months' format to total months
    """
    df = df.copy()
    
    # Create function to extract years and months
    def extract_months(x):
        if pd.isna(x) or x == 'NA':
            return np.nan
        
        # Extract years and months using string operations
        years = 0
        months = 0
        
        if 'Years' in str(x):
            years = int(str(x).split('Years')[0].strip())
        if 'Months' in str(x):
            months = int(str(x).split('Months')[0].split('and')[-1].strip())
            
        return years * 12 + months

    # Apply conversion
    df[target_col] = df[source_col].apply(extract_months)
    df = df.drop(columns=[source_col])
    
    return df

# Apply transformation
df_raw = convert_credit_history_age_to_months(df_raw)
print("\nCredit History Age conversion results:")
print(df_raw['Credit_History_Age_Months'].describe())


Credit History Age conversion results:
count    90970.000000
mean       221.195405
std         99.741364
min          1.000000
25%        144.000000
50%        219.000000
75%        302.000000
max        404.000000
Name: Credit_History_Age_Months, dtype: float64


In [42]:
df_raw.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_History_Age_Months
count,97224.000000,1.000000e+05,1.000000e+05,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,92998.000000,97909.000000,98035.000000,100000.000000,100000.000000,95521.000000,9.880000e+04,90970.000000
mean,33.323274,1.764157e+05,5.743259e+03,17.091280,22.47443,72.466040,3.009960,21.068780,30.923342,10.389025,27.754251,32.285173,1403.118217,637.412998,-3.036437e+22,221.195405
std,10.778909,1.429618e+06,4.581469e+04,117.404834,129.05741,466.422621,62.647879,14.860104,226.031892,6.789496,193.177339,5.116875,8306.041270,2043.319327,3.181295e+24,99.741364
min,14.000000,7.005930e+03,3.036454e+02,-1.000000,0.00000,1.000000,-100.000000,-5.000000,-3.000000,-6.490000,0.000000,20.000000,0.000000,0.000000,-3.333333e+26,1.000000
25%,24.000000,1.945750e+04,1.625793e+03,3.000000,4.00000,8.000000,1.000000,10.000000,9.000000,5.320000,3.000000,28.052567,30.306660,74.534002,2.700922e+02,144.000000
50%,33.000000,3.757861e+04,3.101372e+03,6.000000,5.00000,13.000000,3.000000,18.000000,14.000000,9.400000,6.000000,32.305784,69.249473,135.925682,3.367192e+02,219.000000
75%,42.000000,7.279092e+04,5.971780e+03,7.000000,7.00000,20.000000,5.000000,28.000000,18.000000,14.870000,9.000000,36.496663,161.224249,265.731733,4.702202e+02,302.000000
max,100.000000,2.419806e+07,1.990380e+06,1798.000000,1499.00000,5797.000000,1496.000000,67.000000,4397.000000,36.970000,2597.000000,50.000000,82331.000000,10000.000000,1.602041e+03,404.000000


# Análise para One-hot encoding

Pensando em colunas com strings, valores categóricos, temos sempre algumas opções para o modelo: não usá-las, dar peso para os valores, transformar em número ou fazer hot encoding.
Aqui decidimos por fazr one-hot encoding para colunas que possuem até 10 valores distintos e uma simples substituição para colunas que tenham mais de 10 valores únicos (devido ao maior pedido de processamento que colunas assim teriam ao passar por OHE).
Fizemos a contagem da variável target abaixo não para aplicar o OHE, mas para já entender um pouco mais se estamos trabalhando com um resultado balanceado ou não.

In [43]:
def show_full_value_counts_and_distinct(df, cols):
    """
    For each column, show:
    1) Number of unique values
    2) Value counts
    """
    for col in cols:
        print(f"\n===== {col} (distinct: {df[col].nunique()}) =====")
        print(df[col].value_counts().sort_values(ascending=False).head(20))

# Columns to analyze
cols_to_count = [
    "Type_of_Loan",
    "Credit_Mix",
    "Payment_of_Min_Amount",
    "Payment_Behaviour",
    "Credit_Score",
    "Month",
    "Occupation"
]

show_full_value_counts_and_distinct(df_raw, cols_to_count)


===== Type_of_Loan (distinct: 6260) =====
Type_of_Loan
Not Specified                               1408
Credit-Builder Loan                         1280
Personal Loan                               1272
Debt Consolidation Loan                     1264
Student Loan                                1240
Payday Loan                                 1200
Mortgage Loan                               1176
Auto Loan                                   1152
Home Equity Loan                            1136
Personal Loan, and Student Loan              320
Not Specified, and Payday Loan               272
Mortgage Loan, and Home Equity Loan          264
Student Loan, and Payday Loan                256
Student Loan, and Credit-Builder Loan        248
Credit-Builder Loan, and Not Specified       248
Payday Loan, and Auto Loan                   240
Payday Loan, and Debt Consolidation Loan     240
Payday Loan, and Student Loan                232
Personal Loan, and Credit-Builder Loan       232
Mortgage Loan

# Executando o One-hot encoding

In [44]:
# Columns for one-hot encoding (those with <=10 unique values)
onehot_cols = ["Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour", "Month"]
# Columns for label encoding
index_only = ["Occupation", "Type_of_Loan"]

def encode_columns(df, index_cols, ohe_cols):
    """
    Apply Label Encoding to index_cols and One-Hot Encoding to ohe_cols
    """
    df = df.copy()
    
    # Label Encoding
    le = LabelEncoder()
    for col in index_cols:
        df[f"{col}_idx"] = le.fit_transform(df[col])
        df = df.drop(columns=[col])
    
    # One-Hot Encoding
    df = pd.get_dummies(df, columns=ohe_cols, prefix=ohe_cols)
    
    return df

# Apply encodings
df_raw = encode_columns(df_raw, index_only, onehot_cols)

print("\nEncoded columns (first few rows):")
encoded_cols = ([f"{col}_idx" for col in index_only] + 
                [col for col in df_raw.columns if any(x in col for x in onehot_cols)])
print(df_raw[encoded_cols].head())

print("\nDataset structure after encoding:")
print(df_raw.info())


Encoded columns (first few rows):
   Occupation_idx  Type_of_Loan_idx  Monthly_Inhand_Salary  Monthly_Balance  \
0              13               128            1824.843333       312.494089   
1              13               128            1592.843333       284.629162   
2              13               128            1592.843333       331.209863   
3              13               128            1592.843333       223.451310   
4              13               128            1824.843333       341.489231   

   Credit_History_Age_Months  Credit_Mix_Bad  Credit_Mix_Good  Credit_Mix_N/A  \
0                      265.0           False            False            True   
1                        NaN           False             True           False   
2                      267.0           False             True           False   
3                      268.0           False             True           False   
4                      269.0           False             True           False   

   

# Save Processed Dataset
Save the transformed and encoded dataset to the processed data folder for further use in modeling.

In [45]:
# Save the processed dataset to CSV
df_raw.to_csv("../data/processed/creditscore_data_processed.csv", index=False)
print("Dataset saved successfully to '../data/processed/creditscore_data_processed.csv'")

Dataset saved successfully to '../data/processed/creditscore_data_processed.csv'
